# DoubleML Analysis: Water Treatment Impact on E.coli Risk

This notebook implements comprehensive Double Machine Learning analysis with:
- Base models (PLR & IRM) with specified covariates
- Extended models (PLR & IRM) with all covariates
- Subsample analysis by RiskSource categories
- LaTeX table outputs for each analysis

In [1]:
import pandas as pd
import numpy as np
import optuna
from doubleml import DoubleMLData, DoubleMLPLR, DoubleMLIRM
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

## 1. Load and Prepare Data

In [2]:
# Load cleaned data
data = pd.read_csv('mics_clean.csv')

print(f"Dataset shape: {data.shape}")
print(f"\nMissing values:\n{data.isnull().sum()[data.isnull().sum() > 0]}")

# Drop rows with NaN values (DoubleML requires complete cases)
data_complete = data.dropna()
print(f"\nComplete cases: {data_complete.shape}")

# Display basic info
data_complete.head()

Dataset shape: (56721, 54)

Missing values:
PipedWater                     3
WellandSpringWater             3
RainandSurfaceWater            3
PurchasedWater                 3
Basic_water_service            2
Limited_water_service          2
Unimproved_water_service       3
Open_defecation             1209
dtype: int64

Complete cases: (55510, 54)


,SomeRiskHome,VeryHighRiskHome,water_treatment,Any_U5,Girls_less_than15,Boys_15or_less,ImprovedWaterSource,PipedWater,WellandSpringWater,RainandSurfaceWater,...,country_cat_Togo,country_cat_Trinidad and Tobago,country_cat_Viet Nam,country_cat_Zimbabwe,WS1_g_Packaged/Bottled water,WS1_g_Piped water,WS1_g_Protected well/spring,WS1_g_Surface/Rain water,WS1_g_Tube/Well/Borehole,WS1_g_Unprotected well/spring
0,1,0,0,1,0,1,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,0,1,0,0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,1,0,1,0,0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0,1,0,1,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,0,0,0,0,0,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 2. Define Variable Sets

In [3]:
# Outcome variables
outcome_vars = ['SomeRiskHome', 'VeryHighRiskHome']

# Treatment variable
treatment_var = 'water_treatment'

# Base model covariates - using what's actually available in the data
# Note: country_cat and WS1_g were one-hot encoded, so we include all their encoded versions
country_cols = [col for col in data_complete.columns if col.startswith('country_cat_')]
ws1g_cols = [col for col in data_complete.columns if col.startswith('WS1_g_')]
region_cols = [col for col in data_complete.columns if col.startswith('Region_')]

base_covariates = ['windex5', 'helevel'] + country_cols + ws1g_cols

# Extended model covariates (all variables except outcomes and treatment)
extended_covariates = [col for col in data_complete.columns 
                       if col not in outcome_vars + [treatment_var]]

# RiskSource categories (these are encoded as integers 0, 1, 2 in the data)
# Mapping: 0='No risk', 1='Moderate to high risk', 2='Very high risk'
risk_categories = {
    0: 'No risk',
    1: 'Moderate to high risk',
    2: 'Very high risk'
}

## 3. Helper Functions

In [4]:
def create_hyperparameter_space():
    """Define hyperparameter search space for XGBoost"""
    return {
        'n_estimators': {'low': 50, 'high': 200, 'step': 25},
        'max_depth': {'low': 2, 'high': 6},
        'min_child_weight': {'low': 1, 'high': 10},
        'subsample': {'low': 0.6, 'high': 0.9},
        'colsample_bytree': {'low': 0.6, 'high': 0.9},
        'learning_rate': {'low': 0.01, 'high': 0.1}
    }

def run_doubleml_model(data, outcome, covariates, model_type='plr', n_trials=50):
    """
    Run DoubleML model with hyperparameter tuning
    
    Parameters:
    -----------
    data : DataFrame
        Complete dataset
    outcome : str
        Outcome variable name
    covariates : list
        List of covariate names
    model_type : str
        'plr' or 'irm'
    n_trials : int
        Number of Optuna trials
        
    Returns:
    --------
    dict : Model results
    """
    
    # Create DoubleML data object
    dml_data = DoubleMLData(
        data=data,
        y_col=outcome,
        d_cols=treatment_var,
        x_cols=covariates
    )
    
    # Initialize XGBoost classifiers
    if model_type == 'plr':
        ml_l = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
        ml_m = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
        model = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m)
        
        # Hyperparameter tuning functions
        def ml_l_params(trial):
            hp = create_hyperparameter_space()
            return {
                'n_estimators': trial.suggest_int('n_estimators', hp['n_estimators']['low'], 
                                                 hp['n_estimators']['high'], step=hp['n_estimators']['step']),
                'max_depth': trial.suggest_int('max_depth', hp['max_depth']['low'], hp['max_depth']['high']),
                'min_child_weight': trial.suggest_int('min_child_weight', hp['min_child_weight']['low'], 
                                                     hp['min_child_weight']['high']),
                'subsample': trial.suggest_float('subsample', hp['subsample']['low'], hp['subsample']['high']),
                'colsample_bytree': trial.suggest_float('colsample_bytree', hp['colsample_bytree']['low'], 
                                                        hp['colsample_bytree']['high']),
                'learning_rate': trial.suggest_float('learning_rate', hp['learning_rate']['low'], 
                                                    hp['learning_rate']['high'], log=True)
            }
        
        def ml_m_params(trial):
            hp = create_hyperparameter_space()
            return {
                'n_estimators': trial.suggest_int('n_estimators', hp['n_estimators']['low'], 
                                                 hp['n_estimators']['high'], step=hp['n_estimators']['step']),
                'max_depth': trial.suggest_int('max_depth', hp['max_depth']['low'], hp['max_depth']['high']),
                'min_child_weight': trial.suggest_int('min_child_weight', hp['min_child_weight']['low'], 
                                                     hp['min_child_weight']['high']),
                'subsample': trial.suggest_float('subsample', hp['subsample']['low'], hp['subsample']['high']),
                'colsample_bytree': trial.suggest_float('colsample_bytree', hp['colsample_bytree']['low'], 
                                                        hp['colsample_bytree']['high']),
                'learning_rate': trial.suggest_float('learning_rate', hp['learning_rate']['low'], 
                                                    hp['learning_rate']['high'], log=True)
            }
        
        param_space = {'ml_l': ml_l_params, 'ml_m': ml_m_params}
        
    else:  # IRM
        ml_g = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
        ml_m = XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42)
        model = DoubleMLIRM(dml_data, ml_g=ml_g, ml_m=ml_m)
        
        def ml_g_params(trial):
            hp = create_hyperparameter_space()
            return {
                'n_estimators': trial.suggest_int('n_estimators', hp['n_estimators']['low'], 
                                                 hp['n_estimators']['high'], step=hp['n_estimators']['step']),
                'max_depth': trial.suggest_int('max_depth', hp['max_depth']['low'], hp['max_depth']['high']),
                'min_child_weight': trial.suggest_int('min_child_weight', hp['min_child_weight']['low'], 
                                                     hp['min_child_weight']['high']),
                'subsample': trial.suggest_float('subsample', hp['subsample']['low'], hp['subsample']['high']),
                'colsample_bytree': trial.suggest_float('colsample_bytree', hp['colsample_bytree']['low'], 
                                                        hp['colsample_bytree']['high']),
                'learning_rate': trial.suggest_float('learning_rate', hp['learning_rate']['low'], 
                                                    hp['learning_rate']['high'], log=True)
            }
        
        def ml_m_params(trial):
            hp = create_hyperparameter_space()
            return {
                'n_estimators': trial.suggest_int('n_estimators', hp['n_estimators']['low'], 
                                                 hp['n_estimators']['high'], step=hp['n_estimators']['step']),
                'max_depth': trial.suggest_int('max_depth', hp['max_depth']['low'], hp['max_depth']['high']),
                'min_child_weight': trial.suggest_int('min_child_weight', hp['min_child_weight']['low'], 
                                                     hp['min_child_weight']['high']),
                'subsample': trial.suggest_float('subsample', hp['subsample']['low'], hp['subsample']['high']),
                'colsample_bytree': trial.suggest_float('colsample_bytree', hp['colsample_bytree']['low'], 
                                                        hp['colsample_bytree']['high']),
                'learning_rate': trial.suggest_float('learning_rate', hp['learning_rate']['low'], 
                                                    hp['learning_rate']['high'], log=True)
            }
        
        param_space = {'ml_g': ml_g_params, 'ml_m': ml_m_params}
    
    # Optimize hyperparameters
    optuna_settings = {
        'n_jobs_optuna': -1,
        'show_progress_bar': True,
        'verbosity': optuna.logging.WARNING,
        'n_trials': n_trials
    }
    
    model.tune_ml_models(ml_param_space=param_space, optuna_settings=optuna_settings)
    
    # Fit the model
    model.fit()
    
    # Extract results
    summary = model.summary
    
    return {
        'model': model,
        'coef': model.coef[0],
        'se': model.se[0],
        'ci_lower': model.confint()['2.5 %'].values[0],
        'ci_upper': model.confint()['97.5 %'].values[0],
        'pval': model.pval[0],
        'n_obs': len(data),
        'summary': summary
    }

def create_results_table(results_dict, title):
    """
    Create formatted results table
    
    Parameters:
    -----------
    results_dict : dict
        Dictionary with model results
    title : str
        Table title
        
    Returns:
    --------
    DataFrame : Formatted results table
    """
    rows = []
    for key, result in results_dict.items():
        rows.append({
            'Model': key,
            'Coefficient': f"{result['coef']:.4f}",
            'Std. Error': f"{result['se']:.4f}",
            '95% CI': f"[{result['ci_lower']:.4f}, {result['ci_upper']:.4f}]",
            'p-value': f"{result['pval']:.4f}",
            'n. obs.': f"{result['n_obs']}",
        })
    
    df = pd.DataFrame(rows)
    print(f"\n{'='*80}")
    print(f"{title.center(80)}")
    print(f"{'='*80}")
    print(df.to_string(index=False))
    print(f"{'='*80}\n")
    
    return df

## 4. Analysis: SomeRiskHome

### 4.1 Base Models (PLR & IRM)

In [5]:
# Run base models for SomeRiskHome
somerisk_base_plr = run_doubleml_model(data_complete, 'SomeRiskHome', base_covariates, 'plr')
somerisk_base_irm = run_doubleml_model(data_complete, 'SomeRiskHome', base_covariates, 'irm')

# Create results table
somerisk_base_results = {
    'PLR': somerisk_base_plr,
    'IRM': somerisk_base_irm
}

somerisk_base_table = create_results_table(
    somerisk_base_results, 
    "SomeRiskHome - Base Models"
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                           SomeRiskHome - Base Models                           
Model Coefficient Std. Error             95% CI p-value n. obs.
  PLR     -0.0691     0.0055 [-0.0799, -0.0583]  0.0000   55510
  IRM     -0.0556     0.0080 [-0.0713, -0.0399]  0.0000   55510



In [6]:
# Export to LaTeX
latex_base_somerisk = somerisk_base_table.to_latex(index=False, caption="SomeRiskHome - Base Models (PLR and IRM)",  label="tab:somerisk_base")

# Save to tables/ folder
with open('tables/somerisk_base.tex', 'w') as f:
    f.write(latex_base_somerisk)

### 4.2 Extended Models (PLR & IRM)

In [7]:
# Run extended models for SomeRiskHome
somerisk_ext_plr = run_doubleml_model(data_complete, 'SomeRiskHome', extended_covariates, 'plr')
somerisk_ext_irm = run_doubleml_model(data_complete, 'SomeRiskHome', extended_covariates, 'irm')

# Create results table
somerisk_ext_results = {
    'PLR': somerisk_ext_plr,
    'IRM': somerisk_ext_irm
}

somerisk_ext_table = create_results_table(
    somerisk_ext_results, 
    "SomeRiskHome - Extended Models"
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                         SomeRiskHome - Extended Models                         
Model Coefficient Std. Error             95% CI p-value n. obs.
  PLR     -0.0870     0.0050 [-0.0968, -0.0772]  0.0000   55510
  IRM     -0.0766     0.0075 [-0.0912, -0.0620]  0.0000   55510



In [8]:
# Export to LaTeX
latex_ext_somerisk = somerisk_ext_table.to_latex(index=False, caption="SomeRiskHome - Extended Models (PLR and IRM)",  label="tab:somerisk_ext")

# Save to tables/ folder
with open('tables/somerisk_ext.tex', 'w') as f:
    f.write(latex_ext_somerisk)

### 4.3 Subsample Analysis by RiskSource

In [9]:
# Subsample analysis for SomeRiskHome
somerisk_subsample_results = {}

for risk_cat in risk_categories:
    print(f"\n{'='*60}")
    print(f"RiskSource: {risk_cat}")
    print(f"{'='*60}")
    
    # Filter data for this RiskSource category
    subsample_data = data_complete[data_complete['RiskSource'] == risk_cat].copy()
    print(f"Sample size: {len(subsample_data)}")
    
    if len(subsample_data) < 100:
        print(f"Warning: Small sample size for {risk_cat}")
    
    # Run all 4 models
    base_plr = run_doubleml_model(subsample_data, 'SomeRiskHome', base_covariates, 'plr', n_trials=50)
    base_irm = run_doubleml_model(subsample_data, 'SomeRiskHome', base_covariates, 'irm', n_trials=50)
    ext_plr = run_doubleml_model(subsample_data, 'SomeRiskHome', extended_covariates, 'plr', n_trials=50)
    ext_irm = run_doubleml_model(subsample_data, 'SomeRiskHome', extended_covariates, 'irm', n_trials=50)
    
    somerisk_subsample_results[risk_cat] = {
        'Base PLR': base_plr,
        'Base IRM': base_irm,
        'Extended PLR': ext_plr,
        'Extended IRM': ext_irm
    }
    
    # Create table for this subsample
    subsample_table = create_results_table(
        somerisk_subsample_results[risk_cat],
        f"SomeRiskHome - RiskSource: {risk_cat}"
    )
    
    # Export to LaTeX
    latex_subsample = subsample_table.to_latex(
        index=False, 
        caption=f"SomeRiskHome - RiskSource: {risk_cat}",
        label=f"tab:somerisk_{risk_categories[risk_cat].replace(' ', '_').lower()}"
    )
    
    # Save to tables/ folder
    risk_name = risk_categories[risk_cat].replace(' ', '_').lower()
    outcome_prefix = 'somerisk' if 'SomeRisk' in f"SomeRiskHome - RiskSource: {risk_cat}" else 'veryhigh'
    filename = f'tables/{outcome_prefix}_{risk_name}.tex'
    with open(filename, 'w') as f:
        f.write(latex_subsample)



RiskSource: 0
Sample size: 23323


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                          SomeRiskHome - RiskSource: 2                          
       Model Coefficient Std. Error             95% CI p-value n. obs.
    Base PLR     -0.0840     0.0068 [-0.0973, -0.0706]  0.0000   11584
    Base IRM     -0.0636     0.0079 [-0.0790, -0.0482]  0.0000   11584
Extended PLR     -0.0823     0.0069 [-0.0959, -0.0688]  0.0000   11584
Extended IRM     -0.0634     0.0077 [-0.0784, -0.0484]  0.0000   11584



## 5. Analysis: VeryHighRiskHome

### 5.1 Base Models (PLR & IRM)

In [10]:
# Run base models for VeryHighRiskHome
veryhigh_base_plr = run_doubleml_model(data_complete, 'VeryHighRiskHome', base_covariates, 'plr')
veryhigh_base_irm = run_doubleml_model(data_complete, 'VeryHighRiskHome', base_covariates, 'irm')

# Create results table
veryhigh_base_results = {
    'PLR': veryhigh_base_plr,
    'IRM': veryhigh_base_irm
}

veryhigh_base_table = create_results_table(
    veryhigh_base_results, 
    "VeryHighRiskHome - Base Models"
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                         VeryHighRiskHome - Base Models                         
Model Coefficient Std. Error             95% CI p-value n. obs.
  PLR     -0.0518     0.0059 [-0.0633, -0.0403]  0.0000   55510
  IRM     -0.0443     0.0082 [-0.0604, -0.0283]  0.0000   55510



In [11]:
# Export to LaTeX
latex_base_veryhigh = veryhigh_base_table.to_latex(index=False, caption="VeryHighRiskHome - Base Models (PLR and IRM)", label="tab:veryhigh_base")

# Save to tables/ folder
with open('tables/veryhigh_base.tex', 'w') as f:
    f.write(latex_base_veryhigh)

### 5.2 Extended Models (PLR & IRM)

In [12]:
# Run extended models for VeryHighRiskHome
veryhigh_ext_plr = run_doubleml_model(data_complete, 'VeryHighRiskHome', extended_covariates, 'plr')
veryhigh_ext_irm = run_doubleml_model(data_complete, 'VeryHighRiskHome', extended_covariates, 'irm')

# Create results table
veryhigh_ext_results = {
    'PLR': veryhigh_ext_plr,
    'IRM': veryhigh_ext_irm
}

veryhigh_ext_table = create_results_table(
    veryhigh_ext_results, 
    "VeryHighRiskHome - Extended Models"
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                       VeryHighRiskHome - Extended Models                       
Model Coefficient Std. Error             95% CI p-value n. obs.
  PLR     -0.0670     0.0053 [-0.0774, -0.0567]  0.0000   55510
  IRM     -0.0625     0.0078 [-0.0779, -0.0471]  0.0000   55510



In [13]:
# Export to LaTeX
latex_ext_veryhigh = veryhigh_ext_table.to_latex(index=False, caption="VeryHighRiskHome - Extended Models (PLR and IRM)", label="tab:veryhigh_ext")

# Save to tables/ folder
with open('tables/veryhigh_ext.tex', 'w') as f:
    f.write(latex_ext_veryhigh)

### 5.3 Subsample Analysis by RiskSource

In [14]:
# Subsample analysis for VeryHighRiskHome
print("Running Subsample Analysis for VeryHighRiskHome...\n")

veryhigh_subsample_results = {}

for risk_cat in risk_categories:
    print(f"\n{'='*60}")
    print(f"RiskSource: {risk_cat}")
    print(f"{'='*60}")
    
    # Filter data for this RiskSource category
    subsample_data = data_complete[data_complete['RiskSource'] == risk_cat].copy()
    print(f"Sample size: {len(subsample_data)}")
    
    if len(subsample_data) < 100:
        print(f"Warning: Small sample size for {risk_cat}")
    
    # Run all 4 models
    base_plr = run_doubleml_model(subsample_data, 'VeryHighRiskHome', base_covariates, 'plr', n_trials=50)
    base_irm = run_doubleml_model(subsample_data, 'VeryHighRiskHome', base_covariates, 'irm', n_trials=50)
    ext_plr = run_doubleml_model(subsample_data, 'VeryHighRiskHome', extended_covariates, 'plr', n_trials=50)
    ext_irm = run_doubleml_model(subsample_data, 'VeryHighRiskHome', extended_covariates, 'irm', n_trials=50)
    
    veryhigh_subsample_results[risk_cat] = {
        'Base PLR': base_plr,
        'Base IRM': base_irm,
        'Extended PLR': ext_plr,
        'Extended IRM': ext_irm
    }
    
    # Create table for this subsample
    subsample_table = create_results_table(
        veryhigh_subsample_results[risk_cat],
        f"VeryHighRiskHome - RiskSource: {risk_cat}"
    )
    
    # Export to LaTeX
    latex_subsample = subsample_table.to_latex(
        index=False, 
        caption=f"VeryHighRiskHome - RiskSource: {risk_cat}",
        label=f"tab:veryhigh_{risk_categories[risk_cat].replace(' ', '_').lower()}"
    )
    
    # Save to tables/ folder
    risk_name = risk_categories[risk_cat].replace(' ', '_').lower()
    outcome_prefix = 'somerisk' if 'SomeRisk' in f"SomeRiskHome - RiskSource: {risk_cat}" else 'veryhigh'
    filename = f'tables/{outcome_prefix}_{risk_name}.tex'
    with open(filename, 'w') as f:
        f.write(latex_subsample)


Running Subsample Analysis for VeryHighRiskHome...


RiskSource: 0
Sample size: 23323


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


                        VeryHighRiskHome - RiskSource: 2                        
       Model Coefficient Std. Error             95% CI p-value n. obs.
    Base PLR     -0.2098     0.0130 [-0.2352, -0.1844]  0.0000   11584
    Base IRM     -0.2145     0.0179 [-0.2495, -0.1794]  0.0000   11584
Extended PLR     -0.2109     0.0131 [-0.2366, -0.1853]  0.0000   11584
Extended IRM     -0.2290     0.0191 [-0.2665, -0.1916]  0.0000   11584

